# Interactive Matplotlib
This example demonstrates how to fit emission spectra using `Moose` and visualize the result with `matplotlib` in interactive mode.

The initial guess of the parameters for the fits can be adjusted using the slider widgets in the minimal UI below.

Once satisfied with the guess, pressing the `Fit` button will execute the `cb_fit` callback.

This callback contains the actual call to fit the data.

The fit procedure can be performed on either of two included sample files in the folder `./N2SPS`:

1. A spectrum of the second positive system (0,0) band, as recorded at the start of a nanosecond repetitively pulsed discharge: `20220302_s01_sig.cam` 

2. A spectrum simulated by [SpecAir](http://www.specair-radiation.net/) as part of the unit test from [RADIS](https://github.com/radis/radis)
    * See also: [the corresponding GitHub page](https://github.com/radis/radis/tree/develop/radis/test/files)
    * [Or the notebook by the RADIS developers](https://github.com/radis/massiveOES-examples/blob/main/massiveOES_fits_Specair_N2C.ipynb) that demonstrates using [MassiveOES](https://bitbucket.org/OES_muni/massiveoes/src/master/)


In [1]:
# %matplotlib widget
%matplotlib ipympl
from pathlib import Path
import numpy as np
import pandas as pd
import lmfit
from functools import partial
import Moose

import matplotlib.pyplot as plt

import ipywidgets as widgets
import panel as pn
import panel.widgets as pnw

pn.extension("ipywidgets")


fs = list(Path("N2SPS").glob("*.*"))

sample = pd.read_csv(
    fs[0], sep="\t", header=1, names=["Wavelength", "I"]
)
sample["Norm"] = (sample["I"] - sample["I"].min()) / (
    sample["I"].max() - sample["I"].min()
)

db = Moose.query_DB("N2CB", (320,450))
model = lmfit.Model(Moose.model_for_fit, sim_db=db)
params = lmfit.create_params(**Moose.default_params)

In [2]:
with plt.ioff():
    fig = plt.figure()
    ax = plt.gca()
line_measured = plt.plot(sample['Wavelength'], sample['Norm'], label='Measured')
line_sim = plt.plot(sample['Wavelength'], model.eval(x=sample['Wavelength'].values, params=params), label='Guess')
line_fit = plt.plot(sample['Wavelength'], np.zeros(sample.shape[0]), label='Fit')
line_residual = plt.plot(sample['Wavelength'], np.zeros(sample.shape[0]), label='Residual')

plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('I (a.u.)')


# Create some widgets to interact with and show the output
look = {'sizing_mode': 'stretch_width'}

wids = {p:pnw.FloatSlider(name = p, start=params[p].min, end=params[p].max, value=params[p].value, step=0.01,**look) for p in params if p in ['mu', 'T_rot', 'T_vib', 'sigma', 'gamma']}
wids_result = {p:pnw.FloatInput(name = p,**look) for p in ['mu', 'T_rot', 'T_vib', 'sigma', 'gamma']}
btn_fit = pnw.Button(name='Fit',button_type='primary', **look)
select_file = pnw.Select(name='Select file', options = fs, **look)

layout = pn.Column(pn.Row(*wids.values()), pn.Row(btn_fit, select_file), pn.Row(*wids_result.values(), **look), height=250,**look)

# Create and register callbacks
def update_simulation(event):
    """Callback to update the preview for finetuning the initial guess"""
    x = line_measured[0].get_data()[0]
    ynew = model.eval(x=x, **{n:w.value for n,w in wids.items()})
    line_sim[0].set_data(x,ynew)
    fig.canvas.draw_idle()

def fit_iter_cb(pars, iteration, resid,*args, **kwargs):
    """Callback to update on fit iterations, so we can see the progress"""
    for p in pars:
        if p in ['mu', 'T_rot', 'T_vib', 'sigma', 'gamma']:
            wids_result[p].value = pars[p].value
    if iteration % 20 ==0:
        x = line_measured[0].get_data()[0]
        ynew = model.eval(x=x, **pars)
        line_fit[0].set_data(x,ynew)
        line_residual[0].set_data(x,resid)

def cb_fit(*args):
    """Callback to start the fitting routine.
    Upon completion it makes the button green, to show the fit completed.
    The fitted parameters are updated in `params` upon completion"""
    btn_fit.button_type='danger'
    x,y = line_measured[0].get_data()
    result = model.fit(y, x=x, params=params, iter_cb= fit_iter_cb, max_nfev=300)
    btn_fit.button_type='success'
    for p in  result.params:
        params[p].value=result.params[p].value
    fit_iter_cb(result.params, 20, result.residual) # force update of plot with result when done 

def read_file(*args):
    df = pd.read_csv(select_file.value, sep='\t', header=1, names=['Wavelength', 'I'])
    df['Norm'] = (df['I']-df['I'].min())/(df['I'].max()-df['I'].min())
    line_measured[0].set_data(df['Wavelength'].values, df['Norm'].values)
    line_sim[0].set_data(df['Wavelength'].values, model.eval(x=df['Wavelength'].values, params=params))
    for line in [line_fit, line_residual]:
        line[0].set_data(df['Wavelength'].values, np.zeros(df.shape[0]))
    plt.xlim(df['Wavelength'].min(), df['Wavelength'].max())

for n,w in wids.items():
    w.param.watch(update_simulation,'value')
btn_fit.on_click(cb_fit)

select_file.param.watch(read_file, 'value')

display(widgets.AppLayout(center=fig.canvas), layout)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

Column(height=250, sizing_mode='stretch_width')
    [0] Row
        [0] FloatSlider(end=0.3, name='sigma', sizing_mode='stretch_width', start=0.0001, step=0.01, value=0.05)
        [1] FloatSlider(end=0.3, name='gamma', sizing_mode='stretch_width', start=0.0001, step=0.01, value=0.05)
        [2] FloatSlider(end=2, name='mu', sizing_mode='stretch_width', start=-2, step=0.01)
        [3] FloatSlider(end=10000, name='T_rot', sizing_mode='stretch_width', start=250, step=0.01, value=1000)
        [4] FloatSlider(end=10000, name='T_vib', sizing_mode='stretch_width', start=250, step=0.01, value=1000)
    [1] Row
        [0] Button(button_type='primary', name='Fit', sizing_mode='stretch_width')
        [1] Select(options=[WindowsPath('N2SPS/202203...], sizing_mode='stretch_width', value=WindowsPath('N2SPS/2022030...)
    [2] Row(sizing_mode='stretch_width')
        [0] FloatInput(name='mu', sizing_mode='stretch_width')
        [1] FloatInput(name='T_rot', sizing_mode='stretch_width')
        [2] FloatInput(name='T_vib', sizing_mode='stretch_width')
        [3] FloatInput(name='sigma', sizing_mode='stretch_width')
        [4] FloatInput(name='gamma', sizing_mode='stretch_width')